In [2]:
import yfinance as yf

In [3]:
aapl = yf.Ticker("AAPL")

In [4]:
data = yf.download("AAPL", interval="1d", start="1999-12-31", end="2014-12-31")

[*********************100%***********************]  1 of 1 completed


In [6]:
data["Close"]

Date
1999-12-30     0.895647
1999-12-31     0.917969
2000-01-03     0.999442
2000-01-04     0.915179
2000-01-05     0.928571
                ...    
2014-12-23    28.135000
2014-12-24    28.002501
2014-12-26    28.497499
2014-12-29    28.477501
2014-12-30    28.129999
Name: Close, Length: 3774, dtype: float64

In [7]:
data = yf.download("YNDX", interval="1d", start="1999-12-31", end="2016-12-31")

[*********************100%***********************]  1 of 1 completed


In [10]:
series = data["Close"]

In [17]:
import pandas as pd
import numpy as np
def transform_to_DOIU(prices):
    arr = pd.DataFrame(prices)[0].pct_change().to_numpy()[1:]
    x = [1]
    for el in arr:
        x.append(x[-1]*(1+el))
    return np.array(x)


In [18]:
tr_series = transform_to_DOIU(series.values)

In [27]:
OUTPUT_WINDOW_SIZE = 30
INPUT_WINDOW_SIZE = 60
prices = series.values

X = []
y = []
for i in range(0, len(prices), OUTPUT_WINDOW_SIZE):

    tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                    + OUTPUT_WINDOW_SIZE])
    y.append(tr_x[-1])
    X.append(tr_x[:INPUT_WINDOW_SIZE])

X = X[:-1]
y = y[:-1]

In [30]:
len(X), len(y)

(47, 47)

In [ ]:
def prepare_data(prices, OUTPUT_WINDOW_SIZE = 30, INPUT_WINDOW_SIZE = 60):
    X = []
    y = []
    for i in range(0, len(prices), OUTPUT_WINDOW_SIZE):

        tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                        + OUTPUT_WINDOW_SIZE])
        y.append(tr_x[-1])
        X.append(tr_x[:INPUT_WINDOW_SIZE])

        X = X[:-1]
        y = y[:-1]

[0.5818743638692778,
 0.7317715565014165,
 0.6752367877228687,
 1.0703812609234398,
 0.9299138272763696,
 1.1980503158379516,
 0.8383017153388045,
 0.8009162581808059,
 1.0367744765143432,
 1.1679957436008244,
 1.1651469233374343,
 0.9638554656038548,
 1.0958663292488058,
 0.9361007087170011,
 1.1060542349589042,
 1.2409108515332197,
 1.59184670313487,
 1.4830638744491766,
 1.2313674416212879,
 1.3598875683168716,
 0.9728671832547442,
 0.7917857581995655,
 0.7107692470211946,
 0.9721767468082602,
 0.9629506429598585,
 0.9806937119470365,
 0.7769209387882162,
 0.6231484576199038,
 0.5316326638517915,
 0.5476190579167941,
 1.068888876173231,
 1.1433203030828911,
 1.004840919407991,
 0.5881451409488754,
 0.7739130655924485,
 1.0993955985081194,
 1.0785525462767729,
 1.0414110401405472,
 1.0423148922270862,
 1.6571668023500838,
 1.5571226691473867,
 1.3349168253292256,
 1.0947102833570543,
 0.8069777848759607,
 0.9141437103215665,
 0.9388991912005755,
 1.1018061661959366]